In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
driver = webdriver.Chrome()
driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
time.sleep(3)


In [3]:
#예외 처리
CAFE_NAME = 'rocksoccer'
BOARD_NAME = 'ADs2'
QUERY = '쿠팡'
driver.get(f'http://m.cafe.daum.net/{CAFE_NAME}/{BOARD_NAME}/search?r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&query={QUERY}')
time.sleep(3)
import pandas as pd
from bs4 import BeautifulSoup

# 본문과 댓글을 위한 별도의 DataFrame 초기화
df_columns_posts = ['num', 'subject', 'nick', 'write_time', 'views', 'url', 'contents']
df_posts = pd.DataFrame(columns=df_columns_posts)

df_columns_comments = ['cmt_num', 'cmt_writer', 'cmt_time', 'cmt_txt']
df_comments = pd.DataFrame(columns=df_columns_comments)

# csv파일 읽기
urls_df = pd.read_csv('ROCKSOCCER_쿠팡_32pages_inner_links.csv', header=None, names=['url_list'])
urls = ["http://" + url if not url.startswith("http://") and not url.startswith("https://") else url for url in urls_df['url_list'].tolist()]

for url in urls:
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    inp_num = url.split('/')[-1].split('?')[0]
    
    subject = soup.body.find('h3', class_='tit_subject')
    # 게시물 정보 추가 (게시물 본문 크롤링 부분에 추가)
    if subject is None:
        print(url, '지워진 게시물입니다.')
        continue
    else:
        subject = subject.get_text(strip=True)
        if soup.body.find('span', class_='txt_subject').find('span', class_='sr_only').get_text() == '작성자':
            nick = soup.body.find('span', class_='sr_only').next_sibling
        else:
            nick = ''
        num_subject = soup.body.find_all('span', class_='num_subject')
        write_time = num_subject[0].get_text()
        views = num_subject[1].get_text()
        contents = soup.body.find('div', id='article').get_text('\n', strip=True)
        
        # 게시물 정보를 df_posts에 추가
        new_row_post = {'num': inp_num, 'subject': subject, 'nick': nick, 'write_time': write_time, 'views': views, 'url': url, 'contents': contents}
        df_posts = pd.concat([df_posts, pd.DataFrame([new_row_post])], ignore_index=True)
    
    
        # 댓글
        cmt_total = soup.body.find('article', id='mArticle').find('span', class_="num_total").get_text()
        driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments'.format(CAFE_NAME, BOARD_NAME, inp_num))
        time.sleep(1)
        # 페이지 숫자를 확인하기 위해 캡쳐
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 페이지 숫자 확인
        paging_nav = soup.body.find('span', id="pagingNav")
        if paging_nav is None:
            cmt_page_max = 0
        else:
            num_pages = paging_nav.find_all('span', class_="num_page")
            if len(num_pages) == 0:
                cmt_page_max = 0
            else:
                cmt_page_max = int(num_pages.pop().get_text())
        
        # 댓글 정보 추가 (댓글 크롤링 부분에 추가)
        for i in range(cmt_page_max):
            j = cmt_page_max - i
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            cmt_all = soup.body.find('ul', id="commentList").find_all('li')
            for k in cmt_all:
                # 기본값을 설정합니다
                cmt_reply = False
                reply_prefix = ''
                
                if k.div.span.find_all('span', class_="txt_bar"):
                    if k['class'] == ['reply_on']:
                        cmt_reply = True
                        reply_prefix = 'ㄴ'
                
                # 삭제된 댓글 처리
                if k.find('strong', class_='tit_info'):
                    if k.find('strong', class_='tit_info').find('span', class_='txt_detail').get_text(strip=True) == '삭제된 댓글입니다.':
                        continue
                
                cmt_num = inp_num + '_' + k['id'].split('comment_')[1]
                cmt_writer_elem = k.div.find('span', class_='username')
                if cmt_writer_elem:
                    cmt_writer = cmt_writer_elem.get_text(strip=True)
                else:
                    continue  # 삭제된 댓글이므로 건너뜁니다.
                    
                cmt_time = k.div.span.find('span', class_="created_at").get_text()
                cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
                
                # 댓글 정보를 df_comments에 추가
                new_row_comment = {'cmt_num': reply_prefix + cmt_num, 'cmt_writer': cmt_writer, 'cmt_time': cmt_time, 'cmt_txt': cmt_txt}
                df_comments = pd.concat([df_comments, pd.DataFrame([new_row_comment])], ignore_index=True)
            
            if j > 1:
                driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments?prev_page={3}&mode=regular&cdepth={4}&page={5}'.format(CAFE_NAME, BOARD_NAME, inp_num, j, '0002100000', (j-1)))
                time.sleep(2)

df_posts.to_csv('posts.csv', index=False)
df_comments.to_csv('comments.csv', index=False)

driver.close()


In [4]:
df_posts

,num,subject,nick,write_time,views,url,contents
0,489560,[기사]토트넘과 바이에른 뮌헨은 오는 8월 3일 서울 월드컵경기장에서 일전을 벌인다...,바이언2,24.05.20,280,http://m.cafe.daum.net/rocksoccer/ADs2/489560?...,https://www.besteleven.com/news/articleView.ht...
1,488894,벌써부터 쿠팡 문제가 너무 많네 내년 epl 걱정됨,King of asia ULSAN,24.05.19,258,http://m.cafe.daum.net/rocksoccer/ADs2/488894?...,스포티비랑 같은 급 같음;;; 벌써부터 방송 문제점 한두번이 아니네;;;
2,488886,강원 울산 경기만 쿠팡 안 열리네요. ㅜㅜ,더범,24.05.19,118,http://m.cafe.daum.net/rocksoccer/ADs2/488886?...,'현재 송출 관련 기술적인 문제로 중계 지연'
3,488874,오잉 해설 현영민이네요 ㄷㄷ,버츠비,24.05.19,36,http://m.cafe.daum.net/rocksoccer/ADs2/488874?...,언제 해설했지\n쿠팡에서는 안 보였던 것 같은데 ㄷㄷ
4,487459,축구 밀어주는 쿠팡플레이 참 좋은데.. 메뉴 개선 좀 안되나,축구부장,24.05.11,422,http://m.cafe.daum.net/rocksoccer/ADs2/487459?...,하아..오늘 K리그 경기들 다시 보려는데 너무 불편하네..
...,...,...,...,...,...,...,...
632,170325,"03.25. 한일전 중계 - MBC, 네이버, 아프리카, 카카오, 쿠팡플레이, 트위...",UAENA,21.03.22,"1,335",http://m.cafe.daum.net/rocksoccer/ADs2/170325?...,출처-\nhttps://www.kfa.or.kr/
633,169673,오늘 쿠팡물류센터에서 노가다 뛰느라,성남전위대장,21.03.21,774,http://m.cafe.daum.net/rocksoccer/ADs2/169673?...,성남경기결과 이제야봤는데 오늘 포항상대로 이겼네요... 와우~! 그래서 5위수성.....
634,135842,오재석 자가격리중 떡순튀 먹어,Cha Bum-Kun,20.12.27,841,http://m.cafe.daum.net/rocksoccer/ADs2/135842?...,쿠팡이츠 극찬했다 알려 ㅋㅋㅋㅋㅋㅋ
635,1235,K리그는 계속 진행하는거죠?,레이팔머,20.05.28,294,http://m.cafe.daum.net/rocksoccer/ADs2/1235?se...,"인천학원강사 때문에 쿠팡물류센터 집단 감염 시작했는데 공공기간 박물관, 미술관등 전..."


In [5]:
df_comments

,cmt_num,cmt_writer,cmt_time,cmt_txt
0,488894_1,더범,24.05.19,중계는 제 때 나와야하는데
1,488886_1,별빛사냥꾼,24.05.19,결과만 알수있겠네요 승리하길 바랍니다
2,488886_2,더범작성자 본인 여부작성자,24.05.19,파란 화면뿐
3,487459_1,더범,24.05.11,"그러게요, 경기 찾기 편하게 고쳐줬으면"
4,487425_1,Royal_Black☆SFC,24.05.11,중계권자 입장에서는 잘라버리는 게 맞긴 하죠. 본인들이 중계권 산 건 투자의 일종인...
...,...,...,...,...
3747,1235_5,전북현대 No.11 조규성,20.05.28,이 관계자는 “선수단 내 확진자가 나오면 해당 팀은 최소 2주 이상 경기 일정을 미...
3748,1235_6,레이팔머작성자 본인 여부작성자,20.05.28,공무원+의료진+다수 국민들이 잘 협조해서 막았는데 소수 몇몇 사람들때문에 ㅠ ㅠ
3749,1235_7,포항|김광석,20.05.28,무관중으로한다면야 중단은없을꺼같네요
3750,1233_2,스크립트작성자 본인 여부작성자,20.05.28,쿠팡 코로나사태가 쉽게 끝나보이지않습니다.. 쿠팡 물류센터 직원수가 몇천명인걸로 알...
